<a href="https://colab.research.google.com/github/ezfesoft/sifreleme_algoritmalari/blob/main/LSB_sifre_saklama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pillow

In [ ]:
from PIL import Image

# Mesajı ikili hale getiren yardımcı fonksiyon
def message_to_bin(message):
    return ''.join(format(ord(char), '08b') for char in message)

# İkili veriyi karaktere dönüştüren yardımcı fonksiyon
def bin_to_message(binary):
    binary_chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    message = ''.join([chr(int(char, 2)) for char in binary_chars])
    return message

# 1. Görüntüye gizli mesaj yerleştirme fonksiyonu
def hide_message(image_path, message, output_path):
    # Görüntüyü açma
    img = Image.open(image_path)
    encoded_img = img.copy()
    width, height = img.size
    pixels = encoded_img.load()

    # Mesajı ikili hale çevirme
    binary_message = message_to_bin(message) + '1111111111111110'  # Mesaj sonuna dur bitleri ekleme (1111111111111110)

    message_index = 0

    # Görüntüdeki piksellerin LSB'sine mesajın ikili karşılıklarını yerleştiriyoruz
    for i in range(height):
        for j in range(width):
            if message_index < len(binary_message):
                r, g, b = pixels[j, i]

                # En düşük anlamlı biti değiştirme
                r = (r & 0xFE) | int(binary_message[message_index])
                message_index += 1

                if message_index < len(binary_message):
                    g = (g & 0xFE) | int(binary_message[message_index])
                    message_index += 1

                if message_index < len(binary_message):
                    b = (b & 0xFE) | int(binary_message[message_index])
                    message_index += 1

                pixels[j, i] = (r, g, b)
            else:
                break

    # Yeni görüntüyü kaydetme
    encoded_img.save(output_path)
    print(f"Mesaj başarıyla {output_path} dosyasına gizlendi.")

# 2. Görüntüden gizli mesajı çıkarma fonksiyonu
def extract_message(image_path):
    # Görüntüyü açma
    img = Image.open(image_path)
    pixels = img.load()

    width, height = img.size
    binary_message = ''

    # Piksellerdeki LSB'lerden mesajı çıkarma
    for i in range(height):
        for j in range(width):
            r, g, b = pixels[j, i]

            binary_message += str(r & 1)
            binary_message += str(g & 1)
            binary_message += str(b & 1)

    # Mesajı ayırma (dur bitleri olan '1111111111111110' ile sonlanıyor)
    message_bits = binary_message.split('1111111111111110')[0]

    return bin_to_message(message_bits)

# Mesaj gizleme ve çıkarma örneği
if __name__ == '__main__':
    # Bir görüntüde mesajı gizleyelim
    hide_message('original_image.png', 'Bu bir test mesajıdır.', 'encoded_image.png')

    # Mesajı gizlenmiş görüntüden çıkaralım
    hidden_message = extract_message('encoded_image.png')
    print(f"Gizli mesaj: {hidden_message}")
